
I have used Jupyter notebook to enable ease of evaluation and ensure that everyone can run it using Google Colab GPU reseources. 
This line mounts my Google drive where the data is stored


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


After mounting my Google drive, I have copied the given data file and unzipped it. 
The user has to change the path to the data folder in their drive

In [ ]:
!cp "/content/drive/MyDrive/ml-engineer-testing-task-data.zip" "/content/data.zip" 
!unzip "/content/data.zip"

The code uses the Arcade Learning Environment(ALE) framework for emulating Breakout from the Atari 2600 games. The Open AI gym is built on top of ALE thus offers native support to ALE.  For more details about the framework please refer to - https://github.com/mgbellemare/Arcade-Learning-Environment 

In [5]:
!pip install ale-py

     |████████████████████████████████| 1.6 MB 5.3 MB/s 


This downloads the ROM files required for ATARI 2600 games. The ROM files have been downloaded from http://www.atarimania.com.  However, ALE does not support all ROM files. The code snippet imports the supported ROMS and enables us to easily import the required ROMs using the import command. 

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!unzip "./rars/ROMS.zip"
!mkdir "./checkpoints"
!ale-import-roms ROMS

Importing the Python libraries. The code uses the OpenCV library to process images. Please refer to -https://github.com/opencv/opencv/releases for additional information about OpenCV

In [7]:
import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import copy
import torch.nn as nn
import torch
import torch.optim as optim
from torch.autograd import Variable
from collections import namedtuple, deque
import cv2
import random
from ale_py import ALEInterface
from copy import deepcopy


The Expert class loads the episode start, actions, observations and rewards of the Expert provided in the data.  

In [8]:
class expert():
  def __init__(self, expert_path) -> None:
      if  expert_path is not None:
          assert os.path.exists(expert_path)
      actions = np.load(expert_path+"actions.npy") 
      # Aligning the provided actions to the actions in ALE
      self.actions = np.where(actions<2,actions,actions+1)
      self.obs= np.load(expert_path+"obs.npy")
      self.rewards=np.load(expert_path+"rewards.npy")
      self.episode_starts=np.load(expert_path+"episode_starts.npy")

The below section has some util functions which help streamline the code and make it easy to read and debug. The functions help the code save and load the checkpoints. It also helps in formatting the tensor. There is also a function which enables the agent perfrom null runs in the environment during evaluation. 

In [9]:
def save_checkpoint(state, checkpoint_dir):
	filename = checkpoint_dir + '/network.pth.tar'
	print("Saving checkpoint at " + filename + " ...")
	torch.save(state, filename)  # save checkpoint
	print("Saved checkpoint.")

def get_checkpoint(checkpoint_dir):
	resume_weights = checkpoint_dir + '/network.pth.tar'
	if torch.cuda.is_available():
		print("Attempting to load Cuda weights...")
		checkpoint = torch.load(resume_weights)
		#print "Loaded weights."
	else:
		print("Attempting to load weights for CPU...")
		# Load GPU model on CPU
		checkpoint = torch.load(resume_weights,
								map_location=lambda storage,
								loc: storage)
		print("Loaded weights.")
	return checkpoint

def long_tensor(input):
	if torch.cuda.is_available():
		return torch.cuda.LongTensor(input)
	else:
		return torch.LongTensor(input)

def float_tensor(input):
	if torch.cuda.is_available():
		return torch.cuda.FloatTensor(input)
	else:
		return torch.FloatTensor(input)

def perform_no_ops(ale, no_op_max, preprocessor, state):
	num_no_ops = np.random.randint(1, no_op_max + 1)
	for _ in range(num_no_ops):
		ale.act(0)
		preprocessor.add(ale.getScreenRGB())
	
	while len(preprocessor.preprocess_stack) < 4:
		ale.act(0)
		preprocessor.add(ale.getScreenRGB())
	
	state.add_frame(preprocessor.preprocess())




The Preprocesser creates a stack of 4 frames. It then throws away the first two and takes an element wise maximum of the other two. Then it resizes the image into 84,84 to ensure that the image is aligned with the data provided in the example. 

Note: The pdf document mentions that the provided images are grayscale. However, the images have 3 RGB channel. Thus, this code provides the flexibility to process both coloured and grayscale input images.



In [10]:
from typing import Sequence
class Preprocessor:

	def __init__(self):
		self.preprocess_stack = deque([], 4)

	def add(self, aleRGB):
		self.preprocess_stack.append(aleRGB)

	'''
	As mentioned in the question, preprocessing takes the maximum pixel
	values of 4 consecutive frames. It then
	grayscales the image, and resizes it to 84x84.
	'''
	def preprocess(self):
		assert len(self.preprocess_stack) == 4
		max_stack = np.maximum(self.preprocess_stack[2], self.preprocess_stack[3])
		if sequential:
			img=self.resize(self.grayscale(max_stack))
			return img
		else:
		  return self.resize(max_stack)

	'''
	Takes in an RGB image and returns a grayscaled
	image.
	'''
	def grayscale(self, img):
		return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

	'''
	Resizes the input to an 84x84 image.
	'''
	def resize(self, image):
		return cv2.resize(image, (84, 84),interpolation=cv2.INTER_LINEAR )

This class is a wrapper for the the Arcade learning environment and defines methods to fetch the observations, take actions and reset the game. It also returns a terminal state once the game is over.

In [11]:

class ALEInterfaceWrapper:
	def __init__(self, repeat_action_probability):
		self.internal_action_repeat_prob = repeat_action_probability
		self.prev_action = 0
		self.ale = ALEInterface()
		'''
		This sets the probability from the default 0.25 to 0.
		It ensures deterministic actions.
		'''
		self.ale.setFloat('repeat_action_probability', repeat_action_probability)

	def getScreenRGB(self):
		return self.ale.getScreenRGB()
		
	def game_over(self):
		return self.ale.game_over()

	def reset_game(self):
		self.ale.reset_game()

	def lives(self):
		return self.ale.lives()

	def getMinimalActionSet(self):
		return self.ale.getMinimalActionSet()

	def setInt(self, key, value):
		self.ale.setInt(key, value)

	def setFloat(self, key, value):
		self.ale.setFloat(key, value)

	def loadROM(self, rom):
		self.ale.loadROM(rom)

	def act(self, action):
		actual_action = action
		return self.ale.act(actual_action)

This class takes the observation from ALE and provides the flexibility to process a sequence of images. This enables the algorithm to create a sequence of output.

In [12]:
class State:	
    def __init__(self, hist_len):
      # Initialize a 1 x hist_len x 84 x 84  state
      self.hist_len = hist_len
      if sequential:
        self.state = np.zeros((1, hist_len, 84, 84), dtype=np.float32)
        self.insertLoc = 0
      else:
        self.state = np.zeros((1,84, 84,3), dtype=np.float32)
      

    def add_frame(self, img):
      if sequential:
        self.state[0, self.insertLoc, ...] = img.astype(np.float32)/255.0
        self.insertLoc = (self.insertLoc + 1) % self.hist_len
      else:
        self.state = img.astype(np.float32)

    def get_state(self):      
      if len(self.state.shape)!= 4:
        self.state= np.expand_dims(self.state,axis=0)
      if sequential: 
         return np.roll(self.state, 0 - self.insertLoc, axis=1)
      return self.state
     

This class creates the trainnig dataset based on the Expert's trajectory. The add frame method enables us to add each state action pair sequentially as per the required dataset size. The dataset size enables the user to control the amount of training data as per the compute.  

In [13]:

Example = namedtuple('Example', 'state action')

class Dataset:

	def __init__(self, size, hist_len):
		self.size = size
		self.hist_len = hist_len
		if sequential:
			self.states = np.empty((size, hist_len, 84, 84), dtype=np.float32)
		else:
			self.states = np.empty((size, 84, 84,3), dtype=np.float32)
		self.actions = np.empty(size, dtype=np.uint8)
		self.index = 0
		self.sample_indices = range(size)
		self.shuffle_indices()
		self.minibatch_index = 0

	def shuffle_indices(self):
		random.shuffle(list(self.sample_indices))

	def clear(self):
		self.actions = np.empty(self.size, dtype=np.uint8)
		if sequential:
			self.states = np.empty((self.size,self.hist_len, 84, 84), dtype=np.uint8)
		else:
			self.states = np.empty((self.size, 84, 84,3), dtype=np.uint8)
		
	def grayscale(self, img):
		return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

	def add_item(self, state, action, episode_starts):
		if self.index == self.size:
			raise ValueError("Dataset is full. Clear dataset before adding anything.")
	 
		path = "/content/ml-engineer-testing-task/data/"
		if episode_starts:
				self.states[0, ...] = self.grayscale(cv2.imread(os.path.join(path,state)))
		self.states[self.index, ...] = self.grayscale(cv2.imread(os.path.join(path,state)))
		self.actions[self.index] = action
		self.index += 1

	def sample_minibatch(self, batch_size):
		batch = []
		for _ in range(batch_size):
			index = self.sample_indices[self.minibatch_index]
			batch.append(Example(state=self.states[index],action=self.actions[index]))
			self.minibatch_index = self.minibatch_index + 1
			if self.minibatch_index >= self.size:
				self.minibatch_index = 0
				self.shuffle_indices()
		return batch

This class defines the Convolutional Neural Network(CNN) which is trained to learn from the expert's trajectory. 

In [14]:
class Network(nn.Module):
	def __init__(self, num_output_actions):
		super(Network, self).__init__()
		if not sequential:
				self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
				nn.init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
		else: 
				self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
				nn.init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
		self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
		nn.init.kaiming_normal_(self.conv2.weight, nonlinearity='relu')
		self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
		nn.init.kaiming_normal_(self.conv3.weight, nonlinearity='relu')
		self.fc1 = nn.Linear(64 * 7 * 7, 512)
		self.output = nn.Linear(512, num_output_actions)

	def forward(self, input):
		if not sequential:
			input= input.permute(0,3,1,2)
			input =input.to(memory_format=torch.channels_last)
		conv1_output = F.relu(self.conv1(input))
		conv2_output = F.relu(self.conv2(conv1_output))
		conv3_output = F.relu(self.conv3(conv2_output)).contiguous()
		fc1_output = F.relu(self.fc1(conv3_output.view(conv3_output.size(0), -1)))	
		output = self.output(fc1_output)
		return conv1_output, conv2_output, conv3_output, fc1_output, output

This is the central class which enables the agent to learn from the demonstration provided by the expert. I have used the Behaviour Cloning imitation learning technique to make the agent learn from the expert.

In [15]:
class Imitator:
	def __init__(self, min_action_set,
				learning_rate,
				alpha,
				min_squared_gradient,
				checkpoint_dir,
				hist_len,
				l2_penalty):
		self.minimal_action_set = min_action_set
		self.network = Network(len(self.minimal_action_set))
		if torch.cuda.is_available():
			print("Initializing Cuda Nets...")
			self.network.cuda()
		self.optimizer = optim.Adam(self.network.parameters(),
		lr=learning_rate, weight_decay=l2_penalty)
		self.checkpoint_directory = checkpoint_dir
		self.losses = []
		self.accs=[]


	def predict(self, state):
		# predict action probabilities
		outputs = self.network(Variable(float_tensor(state)))
		vals = outputs[len(outputs) - 1].data.cpu().numpy()
		return vals

	def get_action(self, state):
		vals = self.predict(state)
		return self.minimal_action_set[np.argmax(vals)]

	
	def compute_labels(self, sample, minibatch_size):
		labels = Variable(long_tensor(minibatch_size))
		actions_taken = [x.action for x in sample]
		action_indices = [self.minimal_action_set.index(x) for x in actions_taken]
		for index in range(len(action_indices)):
			labels[index] = action_indices[index]
		return labels

	def get_loss(self, outputs, labels):
		return nn.CrossEntropyLoss()(outputs, labels)

	def train(self, dataset, minibatch_size):
		sample = dataset.sample_minibatch(minibatch_size)
		state = Variable(float_tensor(np.stack([np.squeeze(x.state) for x in sample])))
		labels = self.compute_labels(sample, minibatch_size)
		self.optimizer.zero_grad()
		activations = self.network(state)
		output = activations[len(activations) - 1]
		loss = self.get_loss(output, labels)
		self.losses.append(loss)
		loss.backward()
		self.optimizer.step()
		correct = (torch.argmax(output, dim=1) == labels).float().sum()
		acc = correct/output.shape[0]
		self.accs.append(acc)


	def checkpoint_network(self):
		print("Checkpointing Weights")
		save_checkpoint({'state_dict': self.network.state_dict()}, self.checkpoint_directory)
		print("Checkpointed.")
	

This class implements methods to evaluate the trained DQN agent. The perfromance is measured for 100 episodes. The agent uses an epsilon delta method to ensure optimum balance between exploration and exploitation. 

In [16]:
class Evaluator:

	def __init__(self, rom, cap_eval_episodes=True, time_limit=60 * 60 * 30 / 4,
				action_repeat=4, hist_len=4, ale_seed=100, action_repeat_prob=0,
				num_eval_episodes=100):
		self.cap_eval_episodes = cap_eval_episodes
		self.time_limit = time_limit
		self.action_repeat = action_repeat
		self.hist_len = hist_len
		self.rom = rom
		self.ale_seed = ale_seed
		self.action_repeat_prob = action_repeat_prob
		self.num_eval_episodes = num_eval_episodes

	def evaluate(self, agent):
		ale = self.setup_eval_env(self.ale_seed, self.action_repeat_prob, self.rom)
		self.eval(ale, agent)

	def setup_eval_env(self, ale_seed, action_repeat_prob, rom):
		ale = ALEInterfaceWrapper(action_repeat_prob)
		#Set the random seed for the ALE

		ale.setInt('random_seed', ale_seed)
		'''
		This sets the probability from the default 0.25 to 0.
		It ensures deterministic actions.
		'''
		ale.setFloat('repeat_action_probability', action_repeat_prob)
		# Load the ROM file
		ale.loadROM(rom)
		return ale

	def eval(self, ale, agent):
		action_set = ale.getMinimalActionSet()
		rewards = []
		for i in range(self.num_eval_episodes):
			ale.reset_game()
			preprocessor = Preprocessor()
			state = State(self.hist_len)
			steps = 0
			perform_no_ops(ale, 10, preprocessor, state)
			episode_reward = 0
			while not ale.game_over() and steps < self.time_limit:
				if np.random.uniform() < 0.1:
					action = np.random.choice(action_set)
				else:
					action = agent.get_action(state.get_state())
					if DEBUG:
						print("action", action)
				for _ in range(self.action_repeat):
					episode_reward += ale.act(action)
					preprocessor.add(ale.getScreenRGB())		 		
				state.add_frame(preprocessor.preprocess())
				steps += 1
			print("Episode " + str(i) + " reward is " + str(episode_reward))
			rewards.append(episode_reward)
		print("Mean reward is: " + str(np.mean(rewards)))

This is the main body of the code which trains the DQN agent as per the hyperparameters provided by the user. The most optimum hyperparameters as per my tuning is given below. First the ALE environement is initialised, then the agent is trained to learn from the demonstration from the expert. Finally, the perfromance of the agent is evaluated in the Breakout game. 

In [17]:
import os
import numpy as np
from pdb import set_trace
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('bmh')


def smooth(losses, run=10):
	new_losses = []
	for i in range(len(losses)):
		new_losses.append(np.mean(losses[max(0, i - 10):i+1]))
	return new_losses

def plot(metric, checkpoint_dir, name):
		#p=plt.plot(smooth(losses, 25))
		fig=plt.figure()
		p=plt.plot(metric)
		plt.xlabel("Update")
		plt.ylabel(f"{name}")
		plt.legend(loc='lower center')
		plt.savefig(os.path.join(checkpoint_dir, f"{name}.png"))
		plt.close(fig)
	
def train(rom,
		ale_seed,
		action_repeat_probability,
		learning_rate,
		alpha,
		min_squared_gradient,
		l2_penalty,
		minibatch_size, 
		hist_len,
		discount,
		checkpoint_dir,
		updates,
		dataset):


	ale = ALEInterfaceWrapper(action_repeat_probability)

	#Set the random seed for the ALE
	ale.setInt('random_seed', ale_seed)

	# Load the ROM file
	ale.loadROM(rom)

	print("Minimal Action set is:")
	print(ale.getMinimalActionSet())


	# create DQN agent
	agent = Imitator(ale.getMinimalActionSet(),
				learning_rate,
				alpha,
				min_squared_gradient,
				checkpoint_dir,
				hist_len,
				l2_penalty)

	print("Beginning training...")
	log_frequency = 1000
	log_num = log_frequency
	update = 1
	while update < updates:
		if update > log_num:
			print(str(update) + " updates completed.")
			if DEBUG:
				print("Loss: {:.3f}..".format(agent.losses[-1].data.cpu().numpy()), "Accuracy: {:.3f}".format(agent.accs[-1].cpu().numpy()))
			log_num += log_frequency
		agent.train(dataset, 32)
		update += 1
	print("Training completed.")
	agent.checkpoint_network()
	losses_list,accs_list = [],[]
	for loss,acc in zip(agent.losses,agent.accs):
		losses_list.append(loss.data.cpu().numpy())
		accs_list.append(acc.cpu().numpy())
	plot(metric=losses_list, checkpoint_dir=checkpoint_dir,name="Loss")
	plot(metric=accs_list, checkpoint_dir=checkpoint_dir,name="Accuracy")
	#Evaluation
	evaluator = Evaluator(rom=rom)
	evaluator.evaluate(agent)

Arcade learning environment provides an easy to use method to import the ROMs.

In [ ]:
from ale_py.roms import Breakout

Initialising all the Hyperparameters. The hyperparameters can be passed using the Argument parser if running the code outside of Colab. However, Colab does not permit the use of argument parser. 
The *DEBUG* flag allows the user to print output at critical junctures of the code which makes it easy to debug the code. 
The *SEQUENTIAL*  flag provides the flexibility to the user to run the framework using the RGB channels without converting the image to Grayscale. The user should put SEQUENTIAL to **FALSE** to run the version with the RGB channels. The usual scenario for process a sequence of 4 frames converted to a grayscale image requires the SEQUENTIAL flag to be set to **TRUE.**

In [29]:
sequential =True 
DEBUG = False
rom = Breakout
ale_seed =123
action_repeat_probability =0.0
dataset_size=20000
updates =10000
minibatch_size = 32
hist_len=4
discount=0.99
learning_rate=0.01
alpha=0.95
min_squared_gradient=0.01
l2_penalty= 0.001
checkpoint_dir="checkpoints"

Loading the Expert demonstration using the Expert class. Then, using the dataset class to load all the demonstrations for further processing. 

In [30]:
RL_expert = expert("/content/ml-engineer-testing-task/data/")
data = Dataset(dataset_size, hist_len)
episode_index_counter = 0
for index in range(dataset_size):
      state = str(RL_expert.obs[index])
      action = RL_expert.actions[index]
      data.add_item(state, action ,episode_starts=RL_expert.episode_starts[index])


if DEBUG:
    path = "/content/ml-engineer-testing-task/data/"
    state = RL_expert.obs[2]
    print(np.max(RL_expert.actions))
    print(np.mean(RL_expert.rewards))
    test_img= cv2.imread(os.path.join(path,state))
    h,w,c = test_img.shape
    print(f"height{h} width{w} channel{c}")
    b,g,r = cv2.split(test_img)
    frame_rgb = cv2.merge((r,g,b))
    %matplotlib inline
    plt.imshow(frame_rgb)

The training method to train the DQN agent to learn from the expert class and then evaluate the performance of the DQN Agent.

In [31]:
train(rom,
		ale_seed,
		action_repeat_probability,
		learning_rate,
		alpha,
		min_squared_gradient,
		l2_penalty,
		minibatch_size, 
		hist_len,
		discount,
		checkpoint_dir,
		updates,
		data)

Minimal Action set is:
[<Action.NOOP: 0>, <Action.FIRE: 1>, <Action.RIGHT: 3>, <Action.LEFT: 4>]
Initializing Cuda Nets...
Beginning training...
1001 updates completed.
2001 updates completed.
3001 updates completed.
4001 updates completed.
5001 updates completed.
6001 updates completed.
7001 updates completed.
8001 updates completed.
9001 updates completed.
Training completed.
Checkpointing Weights
Saving checkpoint at checkpoints/network.pth.tar ...
Saved checkpoint.
Checkpointed.


No handles with labels found to put in legend.
No handles with labels found to put in legend.


Episode 0 reward is 11
Episode 1 reward is 11
Episode 2 reward is 11
Episode 3 reward is 0
Episode 4 reward is 4
Episode 5 reward is 8
Episode 6 reward is 0
Episode 7 reward is 0
Episode 8 reward is 12
Episode 9 reward is 11
Episode 10 reward is 8
Episode 11 reward is 0
Episode 12 reward is 11
Episode 13 reward is 0
Episode 14 reward is 11
Episode 15 reward is 11
Episode 16 reward is 11
Episode 17 reward is 0
Episode 18 reward is 11
Episode 19 reward is 11
Episode 20 reward is 11
Episode 21 reward is 8
Episode 22 reward is 11
Episode 23 reward is 0
Episode 24 reward is 0
Episode 25 reward is 0
Episode 26 reward is 11
Episode 27 reward is 0
Episode 28 reward is 0
Episode 29 reward is 11
Episode 30 reward is 0
Episode 31 reward is 11
Episode 32 reward is 11
Episode 33 reward is 0
Episode 34 reward is 8
Episode 35 reward is 0
Episode 36 reward is 11
Episode 37 reward is 11
Episode 38 reward is 11
Episode 39 reward is 0
Episode 40 reward is 11
Episode 41 reward is 11
Episode 42 reward is 0